In [17]:
import pandas as pd
import re
import requests 
import pyodbc
from bs4 import BeautifulSoup
from string import ascii_uppercase
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


In [18]:
productionDB = True
year = 2022
currentProjectionWeek = 1

connection = ''
conn1 = ''
conn2 = ''

if productionDB:
    connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
    conn1 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
    conn2 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
else:
    connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')
    conn1 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')
    conn2 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')


In [19]:
def GetPlayerId(playerName):
    cursor = conn1.cursor()
    cursor.execute("SELECT Id FROM Players WHERE [ProReferenceKey] = ?", playerName)
    playerId = 0
    for row in cursor.fetchall():
        playerId = row[0]
        return playerId

In [20]:
def GetTeamId(team):
    return {
        'ARI':1,
        'CRD':1,
        'ATL':2,
        'BAL':3,
        'RAV':3,
        'BUF':4,
        'CAR':5,
        'CHI':6,
        'CIN':7,
        'CLE':8,
        'DAL':9,
        'DEN':10,
        'DET':11,
        'GNB':12,
        'HOU':13,
        'HTX':13,
        'IND':14,
        'CLT':14,
        'JAX':15,
        'KAN':16,
        'LVR':17,
        'RAI':17,
        'LAC':18,
        'SDG':18,
        'LAR':19,
        'RAM':19,
        'MIA':20,
        'MIN':21,
        'NWE':22,
        'NOR':23,
        'NYG':24,
        'NYJ':25,
        'PHI':26,
        'PIT':27,
        'SFO':28,
        'SEA':29,
        'TAM':30,
        'TEN':31,
        'OTI':31,
        'WAS':32,
        '':'',
        '2TM':''
    }[team]

In [21]:
def GetTeamCode(teamId):
    return {
        1:'ARI',
        1:'CRD',
        2:'ATL',
        3:'BAL',
        3:'RAV',
        4:'BUF',
        5:'CAR',
        6:'CHI',
        7:'CIN',
        8:'CLE',
        9:'DAL',
        10:'DEN',
        11:'DET',
        12:'GNB',
        13:'HOU',
        13:'HTX',
        14:'IND',
        14:'CLT',
        15:'JAX',
        16:'KAN',
        17:'LVR',
        17:'RAI',
        18:'LAC',
        18:'SDG',
        19:'LAR',
        19:'RAM',
        20:'MIA',
        21:'MIN',
        22:'NWE',
        23:'NOR',
        24:'NYG',
        25:'NYJ',
        26:'PHI',
        27:'PIT',
        28:'SFO',
        29:'SEA',
        30:'TAM',
        31:'TEN',
        31:'OTI',
        32:'WAS'
    }[teamId]

In [22]:
positionList = ['QB', 'RB', 'WR', 'TE', 'K']

In [23]:
# NFL Team ID
cursor = connection.cursor()
missingPlayers = []
for i in range(1,33):
    team = GetTeamCode(i).lower()
    print(team)
    url = 'https://www.pro-football-reference.com/teams/{}/2022_roster.htm'.format(team)
    
    driver = webdriver.Chrome(executable_path=r"C:/chromedriver.exe")
    driver.get(url)

    playerTable = driver.find_elements(By.ID, 'roster')
    if playerTable is not None:
        
        cursor.execute("UPDATE Players SET [NFLTeamId] = Null WHERE NFLTeamId = ? AND Id > 0",i)
        cursor.commit()
        playerRows = playerTable[0].find_elements(By.TAG_NAME, 'tbody')[0].find_elements(By.TAG_NAME,'tr')
        for player in playerRows:
            cols = player.find_elements(By.TAG_NAME, 'td')
            if len(cols) > 0:
                name = cols[0].text
                position = cols[2].text
                if '/' in position:
                    position = position.split('/')[0]
                if position in positionList:
                    try:
                        playerAnchor = player.find_element(By.TAG_NAME, 'a')
                        if playerAnchor is None:
                            continue
                        playerHref =playerAnchor.get_attribute('href') 
                        playerKey = playerHref.split('/')[5].split('.')[0]
                        playerId = 0
                        if playerKey != '':
                            playerId = GetPlayerId(playerKey)
                        if playerId is None:
                            print('No Player found for {}'.format(name))
                            
                            height = cols[6].text
                            weight = cols[5].text
                            dob = cols[8].text
                            rookie = cols[9].text == 'Rook'

                            missingPlayers.append([name, playerHref, playerKey, height, weight, dob, rookie, i])
                            continue
                        else:
                            cursor.execute("UPDATE Players SET [NFLTeamId] = ? WHERE Id = ?",i,playerId)
                            cursor.commit()
                            print("Player: {} ({}) - {}".format(name, playerId, position))
                    except:
                        continue
    driver.close()
                
cursor.close()    

crd


C:\Users\Micha\AppData\Local\Temp\ipykernel_40368\2972354347.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:/chromedriver.exe")


Player: Stephen Anderson (37) - TE
Player: Eno Benjamin (1520) - RB
Player: Marquise Brown (171) - WR
Player: James Conner (272) - RB
Player: Greg Dortch (361) - WR
Player: Zach Ertz (402) - TE
Player: A.J. Green (525) - WR
Player: Keaontay Ingram (1668) - RB
Player: Andy Isabella (690) - WR
Player: Trey McBride (1620) - TE
Player: Colt McCoy (893) - QB
Player: Rondale Moore (1538) - WR
Player: Kyler Murray (974) - QB
Player: Matt Prater (1081) - K
Player: Jonathan Ward (1390) - RB
No Player found for Antoine Wesley
Player: Darrel Williams (1448) - RB
Player: Maxx Williams (1458) - TE
Player: Deon Yelder (1502) - TE
Player: DeAndre Hopkins (SUS) (661) - WR
atl
Player: Tyler Allgeier (1654) - RB
No Player found for Jared Bernhardt
Player: Damiere Byrd (201) - WR
Player: Bryan Edwards (387) - WR
No Player found for Feleipe Franks
No Player found for Parker Hesse
Player: KhaDarel Hodge (640) - WR
Player: Younghoe Koo (794) - K
Player: Drake London (1604) - WR
Player: Marcus Mariota (865) 

In [24]:
cursor = connection.cursor()
for player in missingPlayers:
    playerUrl = player[1]
    nameLink = playerUrl.replace('https://www.pro-football-reference.com', '')
    playerPage = requests.get(playerUrl)
    playerSoup = BeautifulSoup(playerPage.text, 'html.parser')
    
    fullName = player[0].replace(' (IR)', '').replace(' (PUP)', '').replace(' (NON)', '')
    names = fullName.split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        last = last + ' ' + name

    jerseyContainer = playerSoup.find("div", {"class": "uni_holder"}).find_all('text');
    jerseyNumber = ''
    if len(jerseyContainer) > 0:
        jerseyNumber = jerseyContainer[-1].get_text()

        
    currentTeam = playerSoup.find("span", {"itemprop": "affiliation"})
    team = player[7]
    birthDate = player[5]
    height = player[3]
    weight = player[4]
    rookie = player [6]

    collegeSection = playerSoup.find_all("p")[5]
    college = collegeSection.find('strong',text = re.compile('College'))
    playerCollege = ''
    if college is not None:
        playerCollege = collegeSection.find_all('a', href=True)[0].get_text()
        
    print("Id: {} Link : {} First: {} Last: {} Team: {} DOB: {} Jersey: {} Rookie: {} Position: {} College: {} Height:{}  Weight: {}".format(player[2],nameLink,first.strip(),last.strip(),team,birthDate,jerseyNumber,rookie,position,playerCollege,height,weight ))
    
    if not team:
        cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName], [FirstName], [LastName], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight], [TradingBlock], [Retired],[WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", player[2],nameLink,fullName,first.strip(),last.strip(),jerseyNumber,rookie,position,playerCollege,birthDate,height,weight,False, False,0 )
        cursor.commit()
    else:
        cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName],[FirstName], [LastName], [NFLTeamId], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight], [TradingBlock], [Retired],[WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", player[2],nameLink,fullName,first.strip(),last.strip(),team,jerseyNumber,rookie,position,playerCollege,birthDate,height,weight,False,False,0 )
        cursor.commit()


cursor.close()


Id: WeslAn01 Link : /players/W/WeslAn01.htm First: Antoine Last: Wesley Team: 1 DOB: 10/22/1997 Jersey: 13 Rookie: False Position: TE College: Texas Tech Height:6-4  Weight: 206
Id: BernJa00 Link : /players/B/BernJa00.htm First: Jared Last: Bernhardt Team: 2 DOB:  Jersey: 83 Rookie: True Position: TE College:  Height:6-1  Weight: 195
Id: FranFe00 Link : /players/F/FranFe00.htm First: Feleipe Last: Franks Team: 2 DOB: 12/22/1997 Jersey: 15 Rookie: False Position: TE College: Florida Height:6-6  Weight: 234
Id: HessPa00 Link : /players/H/HessPa00.htm First: Parker Last: Hesse Team: 2 DOB: 5/26/1995 Jersey: 46 Rookie: False Position: TE College: Iowa Height:6-3  Weight: 261
Id: BoldSl00 Link : /players/B/BoldSl00.htm First: Slade Last: Bolden Team: 3 DOB:  Jersey: 82 Rookie: True Position: TE College: Alabama Height:  Weight: 191
Id: BridSh00 Link : /players/B/BridSh00.htm First: Shemar Last: Bridges Team: 3 DOB: 12/26/1997 Jersey: 85 Rookie: True Position: TE College:  Height:6-4  Weight

In [25]:
# Retired, JerseyNumber

cursor = connection.cursor()

for c in ascii_uppercase:
    url = 'https://www.pro-football-reference.com/players/{}/'.format(c)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    players = soup.find("div", {"id": "div_players"}).find_all('p')
    for player in players:
        lastActiveYear = player.get_text().split('-')[-1]
        if (int(lastActiveYear) < 2015):
            continue
        retired = True
        boldPlayer = player.find_all('b')
        if len(boldPlayer) > 0:
            retired = False
            
        
        nameLink = player.find("a", href=True)
        playerRef = nameLink['href'].split('/')[3].split('.')[0]
        
        position = player.get_text().split('(')[1][0:-1].split(')')[0]
        if position in positionList:
            playerUrl = 'https://www.pro-football-reference.com{}'.format(nameLink['href'])
            playerPage = requests.get(playerUrl)
            playerSoup = BeautifulSoup(playerPage.text, 'html.parser')
            jerseyContainer = playerSoup.find("div", {"class": "uni_holder"}).find_all('text');
            jerseyNumber = ''
            
            if len(jerseyContainer) > 0:
                jerseyNumber = jerseyContainer[-1].get_text()
                
            currentTeam = playerSoup.find("span", {"itemprop": "affiliation"})
            team = ''
            if currentTeam is not None:
                t = currentTeam.find("a", href=True)['href'].split('/')[2]
                team = GetTeamId(t.upper())
                
            cursor.execute("SELECT Id FROM Players WHERE [ProReferenceKey] = ?", playerRef)
            for row in cursor.fetchall():
                playerId = row[0]
            if not team:
                print("Id:{} Jersey: {} Retired: {}".format(playerId,jerseyNumber,retired))
                cursor.execute("UPDATE Players SET [JerseyNumber] = ?,[Retired] = ? WHERE Id = ?",jerseyNumber,retired,playerId)
                cursor.commit()
            else:
                print("Id:{} Jersey: {} Retired: {} Team: {}".format(playerId,jerseyNumber,retired,team))
                cursor.execute("UPDATE Players SET [JerseyNumber] = ?,[Retired] = ?,[NFLTeamId] = ? WHERE Id = ?",jerseyNumber,retired,team,playerId)
                cursor.commit()


cursor.close()


Id:5 Jersey: 10 Retired: True
Id:6 Jersey: 22 Retired: False
Id:7 Jersey: 17 Retired: False
Id:8 Jersey: 89 Retired: False
Id:9 Jersey: 36 Retired: False
Id:10 Jersey: 13 Retired: False
Id:11 Jersey: 45 Retired: True
Id:12 Jersey: 84 Retired: False
Id:13 Jersey: 15 Retired: False
Id:14 Jersey: 19 Retired: True
Id:15 Jersey: 26 Retired: False
Id:16 Jersey: 81 Retired: True
Id:17 Jersey: 11 Retired: False
Id:18 Jersey: 28 Retired: True
Id:19 Jersey: 16 Retired: True
Id:20 Jersey: 3 Retired: False
Id:20 Jersey: 84 Retired: False
Id:21 Jersey: 88 Retired: False
Id:22 Jersey: 15 Retired: True
Id:23 Jersey: 81 Retired: False
Id:24 Jersey: 8 Retired: False
Id:25 Jersey: 83 Retired: True
Id:26 Jersey: 37 Retired: True
Id:27 Jersey: 17 Retired: False
Id:28 Jersey: 13 Retired: False
Id:29 Jersey: 3 Retired: False
Id:31 Jersey: 18 Retired: False
Id:32 Jersey: 88 Retired: True
Id:33 Jersey: 89 Retired: False
Id:33 Jersey: 6 Retired: False
Id:34 Jersey: 26 Retired: True
Id:35 Jersey: 3 Retired: Tru

In [ ]:
def BuildSeasonStats(year, seasonId):
    url = 'https://www.pro-football-reference.com/years/{}/passing.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    passingCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            passingCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in passingCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            comp = cols[7].get_text()
            attempts = cols[8].get_text()
            yards = cols[10].get_text()
            td = cols[11].get_text()
            inter = cols[13].get_text()
            qbr = cols[22].get_text()

            if isinstance(playerId, int):
                statId = -1
                passingCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in passingCursor.fetchall():
                    statId = row[0]
                if statId > 0:
                    #update
                    passingCursor.execute("UPDATE [SeasonPlayerStats] SET [GamesPlayed] = ?, [GamesStarted] = ?, [PassYards] = ?, [Completions] = ?,[PassTDs] = ?,[Interceptions] = ?,[PassAttempts] = ?, [QBRating] = ? WHERE [SeasonId] = ? AND [PlayerId] = ?", gamesPlayed,gamesStarted,yards,comp,td,inter,attempts,qbr,seasonId, playerId,)
                    passingCursor.commit()
                else:
                    #add
                    passingCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId], [GamesPlayed], [GamesStarted], [PassYards], [Completions],[PassTDs],[Interceptions],[PassAttempts],[QBRating]) VALUES (?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesPlayed,gamesStarted,yards,comp,td,inter,attempts,qbr)
                    passingCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Comp: {}  Attempt: {}  Yards: {} TDs: {} INTs: {} QBR: {}".format(name.strip(), playerId,gamesStarted,gamesPlayed,comp,attempts,yards,td,inter,qbr))

    passingCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/rushing.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    rushingCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            rushingCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in rushingCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            attempts = cols[6].get_text()
            yards = cols[7].get_text()
            td = cols[8].get_text()
            average = cols[12].get_text()
            fumbles = cols[13].get_text()

            if isinstance(playerId, int):
                statId = -1
                rushingCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in rushingCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    rushingCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [RushAttempts] = ?,[RushYards] = ?,[RushTDs] = ?,[Fumbles] = ?,[RushAverage] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,attempts,yards,td,fumbles,average,seasonId, playerId)
                    rushingCursor.commit()
                else:
    #                 add
                    rushingCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [RushAttempts],[RushYards],[RushTDs],[Fumbles],[RushAverage]) VALUES (?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,attempts,yards,td,fumbles,average)
                    rushingCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Rush Attempts: {} Yards: {} TDs: {} Fumbles: {} Average: {}".format(name.strip(), playerId,gamesStarted,gamesPlayed,attempts,yards,td,fumbles,average))

    rushingCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/receiving.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    recCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            recCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in recCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            targets = cols[6].get_text()
            rec = cols[7].get_text()
            yards = cols[9].get_text()
            td = cols[11].get_text()
            average = cols[16].get_text()
            fumbles = cols[17].get_text()

            if isinstance(playerId, int):
                statId = -1
                recCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in recCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    recCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [Targets] = ?, [Receptions] = ?, [RecYards] = ?,[RecTDs] = ?,[Fumbles] = ?,[RecAverage] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,targets,rec,yards,td,fumbles,average,seasonId, playerId)
                    recCursor.commit()
                else:
    #                 add
                    recCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [Targets],[Receptions],[RecYards],[RecTDs],[Fumbles],[RecAverage]) VALUES (?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,targets,rec,yards,td,fumbles,average)
                    recCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Targets: {} Catches: {} Yards: {} TDs: {} Fumbles: {} Average: {}".format(name.strip(), playerId,gamesStarted,gamesPlayed,targets,rec,yards,td,fumbles,average))

    recCursor.close()
    

    url = 'https://www.pro-football-reference.com/years/{}/returns.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    returnCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            returnCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in returnCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            prAttempts = cols[6].get_text() 
            prYards = cols[7].get_text()
            prTDs = cols[8].get_text()

            krAttempts = cols[11].get_text() 
            krYards = cols[12].get_text()
            krTDs = cols[13].get_text()

            if isinstance(playerId, int):
                statId = -1
                returnCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in returnCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    returnCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [PRAttempted] = ?,[PRYards] = ?,[PRTDs] = ?,[KRAttempted] = ?,[KRYards] = ?, [KRTDs] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,prAttempts,prYards,prTDs,krAttempts,krYards,krTDs, seasonId, playerId)
                    returnCursor.commit()
                else:
    #                 add
                    returnCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [PRAttempted],[PRYards],[PRTDs],[KRAttempted],[KRYards], [KRTDs]) VALUES (?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,prAttempts,prYards,prTDs,krAttempts,krYards,krTDs)
                    returnCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Punts: {} Attempts, {} Yards, {} TDs  Kick: {} Attempts, {} Yards, {} TDs".format(name.strip(), playerId,gamesStarted,gamesPlayed,prAttempts,prYards,prTDs,krAttempts,krYards,krTDs))

    returnCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/scoring.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    scoringCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            scoringCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in scoringCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()
            made = cols[14].get_text()

            if isinstance(playerId, int):
                statId = -1
                scoringCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in scoringCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    scoringCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [TwoPointConv] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,made, seasonId, playerId)
                    scoringCursor.commit()
                else:
    #                 add
                    scoringCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [TwoPointConv]) VALUES (?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,made)
                    scoringCursor.commit()
            print("Name: {} ({}) Games ({}/{}) - Made:{}".format(name.strip(),playerId,gamesStarted,gamesPlayed,made))

    scoringCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/kicking.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    kickingCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            kickingCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in kickingCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            fgAttempt = cols[16].get_text()
            fgMade = cols[17].get_text()

            xpAttempt = cols[20].get_text()
            xpMade = cols[21].get_text()

            fg50Att = cols[14].get_text()
            fg50Made = cols[15].get_text()

            fg40Att = cols[12].get_text()
            fg40Made = cols[13].get_text()

            fg30Att = cols[10].get_text()
            fg30Made = cols[11].get_text()        

            fg20Att = cols[8].get_text()
            fg20Made = cols[9].get_text()        

            fg19Att = cols[6].get_text()
            fg19Made = cols[7].get_text()

            if isinstance(playerId, int):
                statId = -1
                kickingCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in kickingCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    kickingCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [FieldGoalsMade] = ?, [FieldGoalsAttempted] = ?, [FieldGoals1to19Attempted] = ?,[FieldGoals1to19Made] = ?,[FieldGoals20to29Attempted] = ?,[FieldGoals20to29Made] = ?,[FieldGoals30to39Attempted] = ?,[FieldGoals30to39Made] = ?,[FieldGoals40to49Attempted] = ?,[FieldGoals40to49Made] = ?,[FieldGoals50PlusAttempted] = ?,[FieldGoals50PlusMade] = ?,[PATAttempted] = ?,[PATMade] = ?  WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,fgMade,fgAttempt,fg19Att,fg19Made,fg20Att,fg20Made,fg30Att,fg30Made,fg40Att,fg40Made,fg50Att,fg50Made,xpAttempt,xpMade,seasonId, playerId)
                    kickingCursor.commit()
                else:
    #                 add
                    kickingCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed],[FieldGoalsMade],[FieldGoalsAttempted],[FieldGoals1to19Attempted],[FieldGoals1to19Made],[FieldGoals20to29Attempted],[FieldGoals20to29Made],[FieldGoals30to39Attempted],[FieldGoals30to39Made],[FieldGoals40to49Attempted],[FieldGoals40to49Made],[FieldGoals50PlusAttempted],[FieldGoals50PlusMade],[PATAttempted] ,[PATMade]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,fgMade,fgAttempt,fg19Att,fg19Made,fg20Att,fg20Made,fg30Att,fg30Made,fg40Att,fg40Made,fg50Att,fg50Made,xpAttempt,xpMade)
                    kickingCursor.commit()
            print("Name: {} ({}) Games ({}/{}) - FG:{}/{}  XP:{}/{} 1-19:{}/{} 20-29:{}/{} 30-39:{}/{} 40-49:{}/{} 50+:{}/{}".format(name.strip(),playerId,gamesStarted,gamesPlayed,fgMade,fgAttempt,xpAttempt,xpMade,fg19Made,fg19Att,fg20Made,fg20Att,fg30Made,fg30Att,fg40Made,fg40Att,fg50Made,fg50Att))

    kickingCursor.close()


In [9]:
def GetTeamId(team):
    return {
        'ARI':1,
        'Arizona Cardinals':1,
        'CRD':1,
        'ATL':2,
        'Atlanta Falcons':2,
        'BAL':3,
        'RAV':3,
        'Baltimore Ravens':3,
        'BUF':4,
        'Buffalo Bills':4,
        'CAR':5,
        'Carolina Panthers':5,
        'CHI':6,
        'Chicago Bears':6,
        'CIN':7,
        'Cincinnati Bengals':7,
        'CLE':8,
        'Cleveland Browns':8,
        'DAL':9,
        'Dallas Cowboys':9,
        'DEN':10,
        'Denver Broncos':10,
        'DET':11,
        'Detroit Lions':11,
        'GB':12,
        'Green Bay Packers':12,
        'GNB':12,
        'HOU':13,
        'HTX':13,
        'Houston Texans':13,
        'IND':14,
        'Indianapolis Colts':14,
        'CLT':14,
        'JAC':15,
        'JAX':15,
        'Jacksonville Jaguars':15,
        'KAN':16,
        'KC':16,
        'Kansas City Chiefs':16,
        'LV':17,
        'LVR':17,
        'RAI':17,
        'Las Vegas Raiders':17,
        'LAC':18,
        'Los Angeles Chargers':18,
        'SDG':18,
        'LAR':19,
        'Los Angeles Rams':19,
        'RAM':19,
        'MIA':20,
        'Miami Dolphins':20,
        'MIN':21,
        'Minnesota Vikings':21,
        'NWE':22,
        'New England Patriots':22,
        'NE':22,
        'NOR':23,
        'New Orleans Saints':23,
        'NO':23,
        'NYG':24,
        'New York Giants':24,
        'NYJ':25,
        'New York Jets':25,
        'PHI':26,
        'Philadelphia Eagles':26,
        'PIT':27,
        'Pittsburgh Steelers':27,
        'SF':28,
        'San Francisco 49ers':28,
        'SFO':28,
        'SEA':29,
        'Seattle Seahawks':29,
        'TB':30,
        'Tampa Bay Buccaneers':30,
        'TAM':30,
        'TEN':31,
        'Tennessee Titans':31,
        'OTI':31,
        'WAS':32,
        'Washington Football Team':32,
        '':'',
        '2TM':''
    }[team]

In [10]:
url = 'https://www.fantasypros.com/nfl/projections/qb.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

passingCursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    passAttempt = float(cols[1].getText().replace(',', ''))
    complete = float(cols[2].getText().replace(',', ''))
    passYards = float(cols[3].getText().replace(',', ''))
    passTD = float(cols[4].getText().replace(',', ''))
    passInt = float(cols[5].getText().replace(',', ''))
    rushAtt = float(cols[6].getText().replace(',', ''))
    rushYards = float(cols[7].getText().replace(',', ''))
    rushTD = float(cols[8].getText().replace(',', ''))
    fumLost = float(cols[9].getText().replace(',', ''))
    
    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'QB' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = passingCursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    
    updateDB = True
    sqlCheck = "SELECT * FROM WeeklyProjectedStats WHERE PlayerId = ? AND SeasonId = 5 AND NFLWeek = ?"
    statCheck = passingCursor.execute(sqlCheck, playerId,currentProjectionWeek)
    for row in statCheck.fetchall():
        print(row[0])
        updateDB = False
        
    if updateDB:
        passingCursor.execute("INSERT INTO WeeklyProjectedStats ([SeasonId], [PlayerId],[NFLWeek],[PassYards], [Completions],[PassTDs],[Interceptions],[PassAttempts],[RushYards],[RushTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", 4, playerId,currentProjectionWeek,passYards,complete,passTD,passInt,passAttempt,rushYards,rushTD,fumLost,0,0)
        passingCursor.commit()

    print(GetTeamId(nflTeam))
    
passingCursor.close()

Josh Allen
4
Patrick Mahomes II
16
Justin Herbert
18
Lamar Jackson
3
Kyler Murray
1
Jalen Hurts
26
Tom Brady
30
Dak Prescott
9
Joe Burrow
7
Aaron Rodgers
12
Matthew Stafford
19
Trey Lance
28
Russell Wilson
10
Kirk Cousins
21
Derek Carr
17
Tua Tagovailoa
20
Ryan Tannehill
31
Trevor Lawrence
15
Justin Fields
6
Daniel Jones
24
Matt Ryan
14
Mac Jones
22
Jameis Winston
23
Carson Wentz
32
Jared Goff
11
Zach Wilson
25
Baker Mayfield
5
Davis Mills
13
Marcus Mariota
2
Mitch Trubisky
27
Drew Lock
29
Jacoby Brissett
8
Deshaun Watson
8
Kenny Pickett
27
Geno Smith
29
Desmond Ridder
2
Andy Dalton
23
Joe Flacco
25
Sam Darnold
5
Mason Rudolph
27
Tyrod Taylor
24
Taysom Hill
23
Tyler Huntley
3
Jimmy Garoppolo
28
Teddy Bridgewater
20
Joshua Dobbs
8
Trevor Siemian
6
Gardner Minshew II
26
Taylor Heinicke
32
Cooper Rush
9
Malik Willis
31
Colt McCoy
1
Josh Johnson
10
Jordan Love
12
Matt Corral
5
Kyle Allen
13
Blaine Gabbert
30
Brian Hoyer
22
C.J. Beathard
15
Kyle Trask
30
Case Keenum
4
Jarrett Stidham
17
Sea

IntegrityError: ('23000', "[23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Violation of PRIMARY KEY constraint 'PK_WeeklyProjectedStats'. Cannot insert duplicate key in object 'dbo.WeeklyProjectedStats'. The duplicate key value is (966, 4, 1). (2627) (SQLExecDirectW); [23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]The statement has been terminated. (3621)")

In [ ]:
url = 'https://www.fantasypros.com/nfl/projections/rb.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    rushAttempt = float(cols[1].getText().replace(',', ''))
    rushYards = float(cols[2].getText().replace(',', ''))
    rushTD = float(cols[3].getText().replace(',', ''))
    rec = float(cols[4].getText().replace(',', ''))
    recYards = float(cols[5].getText().replace(',', ''))
    recTD = float(cols[6].getText().replace(',', ''))
    fumbles = float(cols[7].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'RB' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rushAttempt, rushYards, rec, fumbles))
    updateDB = True
    sqlCheck ="SELECT * FROM WeeklyProjectedStats WHERE PlayerId = ? AND SeasonId = 4 AND NFLWeek = ?"
    statCheck = cursor.execute(sqlCheck, playerId,currentProjectionWeek)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO WeeklyProjectedStats ([SeasonId], [PlayerId],[NFLWeek],[RushAttempts], [RushYards],[RushTDs],[Receptions],[RecYards],[RecTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", 4, playerId,currentProjectionWeek,rushAttempt,rushYards,rushTD,rec,recYards,recTD,fumbles,0,0)
        cursor.commit()
    print(GetTeamId(nflTeam))
    
cursor.close()

In [ ]:
url = 'https://www.fantasypros.com/nfl/projections/wr.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    rec = float(cols[1].getText().replace(',', ''))
    recYards = float(cols[2].getText().replace(',', ''))
    recTD = float(cols[3].getText().replace(',', ''))
    rushAttempt = float(cols[4].getText().replace(',', ''))
    rushYards = float(cols[5].getText().replace(',', ''))
    rushTD = float(cols[6].getText().replace(',', ''))

    fumbles = float(cols[7].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'WR' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rec, recYards, recTD, fumbles))
    updateDB = True
    sqlCheck ="SELECT * FROM WeeklyProjectedStats WHERE PlayerId = ? AND SeasonId = 4 AND NFLWeek = ?"
    statCheck = cursor.execute(sqlCheck, playerId,currentProjectionWeek)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO WeeklyProjectedStats ([SeasonId], [PlayerId],[NFLWeek],[RushAttempts], [RushYards],[RushTDs],[Receptions],[RecYards],[RecTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", 4, playerId,currentProjectionWeek,rushAttempt,rushYards,rushTD,rec,recYards,recTD,fumbles,0,0)
        cursor.commit()
    print(GetTeamId(nflTeam))
    
cursor.close()

In [ ]:
url = 'https://www.fantasypros.com/nfl/projections/te.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    rec = float(cols[1].getText().replace(',', ''))
    recYards = float(cols[2].getText().replace(',', ''))
    recTD = float(cols[3].getText().replace(',', ''))

    fumbles = float(cols[4].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'TE' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rec, recYards, recTD, fumbles))
    updateDB = True
    sqlCheck ="SELECT * FROM WeeklyProjectedStats WHERE PlayerId = ? AND SeasonId = 4 AND NFLWeek = ?"
    statCheck = cursor.execute(sqlCheck, playerId,currentProjectionWeek)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO WeeklyProjectedStats ([SeasonId], [PlayerId],[NFLWeek],[Receptions],[RecYards],[RecTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?)", 4, playerId,currentProjectionWeek,rec,recYards,recTD,fumbles,0,0)
        cursor.commit()

    print(GetTeamId(nflTeam))
    
cursor.close()

In [ ]:
url = 'https://www.fantasypros.com/nfl/projections/k.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    fg = float(cols[1].getText().replace(',', ''))
    fga = float(cols[2].getText().replace(',', ''))
    xp = float(cols[3].getText().replace(',', ''))

    points = float(cols[4].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'K' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rec, recYards, recTD, fumbles))
    updateDB = True
    sqlCheck ="SELECT * FROM WeeklyProjectedStats WHERE PlayerId = ? AND SeasonId = 4 AND NFLWeek = ?"
    statCheck = cursor.execute(sqlCheck, playerId,currentProjectionWeek)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO WeeklyProjectedStats ([SeasonId], [PlayerId],[NFLWeek],[FieldGoalsMade],[FieldGoalsAttempted],[PATMade],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?)", 4, playerId,currentProjectionWeek,fg,fga,xp,points,0)
        cursor.commit()

    print(GetTeamId(nflTeam))
    
cursor.close()

In [ ]:
url = 'https://www.fantasypros.com/nfl/projections/dst.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()

    sacks = float(cols[1].getText().replace(',', ''))
    dInt = float(cols[2].getText().replace(',', ''))
    fr = float(cols[3].getText().replace(',', ''))
    td = float(cols[5].getText().replace(',', ''))
    safety = float(cols[6].getText().replace(',', ''))
    pa = float(cols[7].getText().replace(',', ''))
    yards = float(cols[8].getText().replace(',', ''))
    points = float(cols[9].getText().replace(',', ''))

    nflTeamId = GetTeamId(fullName) * -1

    playerId = nflTeamId


    updateDB = True
    sqlCheck ="SELECT * FROM WeeklyProjectedStats WHERE PlayerId = ? AND SeasonId = 4 AND NFLWeek = ?"
    statCheck = cursor.execute(sqlCheck, playerId,currentProjectionWeek)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        print("{} {} {} {}".format(fullName, nflTeamId, sacks, td))
        cursor.execute("INSERT INTO WeeklyProjectedStats ([SeasonId], [PlayerId],[NFLWeek],[Sacks],[DefensiveInt],[DefensiveFumbleRecovery],[DefensiveTD],[Safety],[PointsAllowed],[YardsAllowed],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", 4, nflTeamId,currentProjectionWeek,sacks,dInt,fr,td,safety,pa,yards,0,points)
        cursor.commit()


    
cursor.close()